In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.data import Field, TabularDataset, BucketIterator

from dataset.mtgcards import RuleText
from utils.preprocess import fields_for_rule_text

import random
import math
import time
import os

In [17]:
SRC, TRG = fields_for_rule_text()
fields = {'src': ('src', SRC), 'trg': ('trg', TRG)}

train_data, valid_data, test_data = RuleText.splits(fields=fields, version='v2.1')

In [18]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)
print(f"Unique tokens in source (en) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (zh) vocabulary: {len(TRG.vocab)}")

for x in random.sample(list(train_data), 3):
    print(x.src, x.trg)

Unique tokens in source (en) vocabulary: 1475
Unique tokens in target (zh) vocabulary: 2306
['counter', 'target', 'spell', '.'] ['反击', '目标', '咒语', '。']
['choose', 'one', '—'] ['选择', '一', '项', '～']
['equip', '{', '3', '}'] ['佩带', '{', '3', '}']


In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_within_batch = True,
    sort_key = lambda x: len(x.src),
    device = device)

tmp = next(iter(train_iterator))
print(tmp)

cpu

[torchtext.legacy.data.batch.Batch of size 128]
	[.src]:('[torch.LongTensor of size 26x128]', '[torch.LongTensor of size 128]')
	[.trg]:[torch.LongTensor of size 36x128]


In [20]:
from models.model4.definition import Encoder, Attention, Decoder, Seq2Seq
from models.model4.train import init_weights, train, evaluate
from utils import count_parameters, train_loop

In [21]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

model.apply(init_weights)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 11,535,874 trainable parameters


In [22]:
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

train_loop(model, optimizer, criterion, train, evaluate,
           train_iterator, valid_iterator, 
           save_path='result/', file_name='model4-rule-v2.1.pt', load_before_train=True)

model will be saved to result/model4-rule-v2.1.pt


KeyboardInterrupt: 

In [23]:
from utils.translate import Translator
from models.model4.definition import beam_search
model.load_state_dict(torch.load('result/model4-rule-v2.1.pt', map_location=torch.device(device)))
T = Translator(SRC, TRG, model, device, beam_search)

In [24]:
data = 'Whenever <1> becomes attached to a creature, for as long as <1> remains attached to it, you may have that creature become a copy of another target creature you control.'
data = 'target creature gets - 1 / - 1 until end of turn .'
ret, prob = T.translate(data, max_len=100)
print(*ret[:3], sep='\n')

['目标', '生物', '得', '-', '1', '/', '-', '1', '直到', '回合', '结束', '。', '<eos>']
['目标', '生物', '得', '得', '-', '1', '/', '-', '1', '直到', '回合', '结束', '。', '<eos>']
['目标', '生物', '得', '1', '/', '-', '1', '直到', '回合', '结束', '。', '<eos>']


In [25]:
from utils import show_samples
long_data = [x for x in test_data.examples if len(x.src) > 20]
print(f'Number of samples: {len(long_data)}')
show_samples(long_data, T, n=100, beam_size=3)

Number of samples: 212
src: [{ 3 } { g } { g } , exile < 7 > from your graveyard : create a 3 / 3 green beast creature token . ] trg = [{3}{g}{g}，从你的坟墓场放逐<7>：将一个3/3绿色野兽衍生生物放进战场。]
{3}{g}{g}，从你的坟墓场放逐<7>：派出一个3/3绿色野兽衍生生物。<eos> 	[probability: 0.14533]
{3}{g}{g}，从你的坟墓场放逐<7>：将一个3/3绿色野兽衍生生物。<eos> 	[probability: 0.09515]
{3}{g}{g}，从你的坟墓场放逐<7>：派出一个3/3绿色野兽衍生生物放进战场。<eos> 	[probability: 0.04418]

src: [whenever one or more loyalty counters are removed from < 3 > , she deals that much damage to target opponent or planeswalker . ] trg = [每当从<3>上移去一个或数个忠诚指示物时，她向目标对手或鹏洛客造成等量的伤害。]
每当一个<3>上移去一个或数个忠诚指示物时，刻拉诺斯对目标对手造成等量的伤害。<eos> 	[probability: 0.00182]
每当一个<3>上移去一个或数个忠诚指示物时，普罗烽斯对目标对手造成等量的伤害。<eos> 	[probability: 0.00093]
每当一个<3>上移去一个或数个忠诚指示物时，<unk>对目标对手造成等量的伤害。<eos> 	[probability: 0.00063]

src: [whenever one or more pirates you control deal combat damage to a player , exile the top card of that player 's library . ] trg = [每当由你操控的一个或数个海盗对任一牌手造成战斗伤害时，放逐该牌手的牌库顶牌。]
每当由你操控的一个或数个对任一牌手造成战斗伤害时，放逐该牌手的牌库顶牌。<eos> 	[p

In [31]:
from dataset.mtgcards import TestSets
from utils import calculate_bleu
from torchtext.legacy.data import Field
from models.card_name_detector.definition import TrainedDetector
from utils.translate import sentencize, CardTranslator

fields = {'src-rule': ('src', Field(tokenize=lambda x: x.split(' '))), 'trg-rule': ('trg', Field())}
test_data = TestSets.load(fields)

D = TrainedDetector()

path: d:\Desktop\mtg-cards-translation\models\card_name_detector


In [36]:
def sentencize(text: str):
    ignore = {' ', '(', ')', '\n'}
    while len(text) and text[0] in ignore:
        text = text[1:]
    if len(text) == 0:
        return []
    
    r = 0
    delims = {'.', '\n', '('}
    ignore = False
    while r < len(text):
        if text[r] == '\"':
            ignore = not ignore
        if not ignore and text[r] in delims:
            break
        r += 1
    
    if r < len(text) and text[r] == '.':
        return [text[:r + 1]] + sentencize(text[r + 1:])
    return [text[:r]] + sentencize(text[r:])
def preprocess(x:str):
    x = D.annotate(x).removeprefix(' ')
    print(f'[after preprocess]:{x}')
    return x
def postprocess(x:str):
    return x.replace('<', '').replace('>', '')

import re
class CTHelper:
    def __init__(self, name_detector, dictionary={}) -> None:
        self.D = name_detector
        self.dictionary = dictionary
    
    def preprocess(self, x:str):
        self.tag2str = {}
        x = D.annotate(x).removeprefix(' ') # x become lowercase after go through detector
        m = re.search('<[^0-9>]+>', x)
        id = 0
        while m:
            l, r = m.span()
            tag = '<' + str(id) + '>'
            self.tag2str[tag] = x[l:r]
            x = x[:l] + tag + x[r:]
            id += 1
            m = re.search('<[^0-9>]+>', x)

        for s in self.dictionary.keys():
            m = re.search(s, x)
            if m:
                tag = '<' + str(id) + '>'
                self.tag2str[tag] = s
                x = x.replace(s, tag)
                id += 1

        print(f'[after preprocess]:{x}')
        return x

    def postprocess(self, x:str):
        print(f'[before postprocess]:{x}')
        for tag, s in self.tag2str.items():
            x = x.replace(tag, self.dictionary[s] if s in self.dictionary else s)
        return x

dic = {}
dic = {'oil':'烁油', 'rebel':'反抗军','compleated':'完化'}
helper = CTHelper(D, dic)
CT = CardTranslator(sentencize, T, preprocess=lambda x: helper.preprocess(x), postprocess=lambda x:helper.postprocess(x))

# example = random.sample(list(test_data), 1)[0]
example = list(test_data)[15]
print(vars(example))
CT.translate(' '.join(example.src))
# for example in random.sample(list(test_data), 10):
#     print(vars(example))
#     CT.translate(' '.join(example.src))

{'src': ['Flying\nNecrosquito', 'enters', 'the', 'battlefield', 'with', 'two', 'oil', 'counters', 'on', 'it.\nNecrosquito', 'gets', '+1/+1', 'for', 'each', 'oil', 'counter', 'on', 'it.\nWhenever', 'another', 'creature', 'or', 'artifact', 'you', 'control', 'is', 'put', 'into', 'a', 'graveyard', 'from', 'the', 'battlefield,', 'put', 'an', 'oil', 'counter', 'on', 'Necrosquito.'], 'trg': ['飞行', '化尸蚊进战场时上面有两个烁油指示物。', '化尸蚊上每有一个烁油指示物，便得+1/+1。', '每当另一个由你操控的生物或神器从战场进入坟墓场时，在化尸蚊上放置一个烁油指示物。']}
[after preprocess]:flying
[before postprocess]:飞行
[after preprocess]:<0> enters the battlefield with two <1> counters on it .
[before postprocess]:<0>进战场时上面有两个+1指示物。
[after preprocess]:<0> gets + 1 / + 1 for each <1> counter on it .
[before postprocess]:<1>上每有一个1指示物，<0>便得+1/+1。
[after preprocess]:whenever another creature or artifact you control is put into a graveyard from the battlefield , put an <1> counter on <0> .
[before postprocess]:每当另一个由你操控的生物或鹏洛客从战场进入坟墓场时，在<0>上放置一个<1>。


'飞行 <necrosquito>进战场时上面有两个+1指示物。 烁油上每有一个1指示物，<necrosquito>便得+1/+1。 每当另一个由你操控的生物或鹏洛客从战场进入坟墓场时，在<necrosquito>上放置一个烁油。'

In [13]:
from utils import calculate_testset_bleu
calculate_testset_bleu(list(test_data)[:100], CT)

  0%|          | 0/100 [00:00<?, ?it/s]

[after preprocess]:whenever another artifact or creature you control is put into a graveyard from the battlefield , put an <1> counter on <0> .
[after preprocess]:{2} {b} , {t} , remove four <1> counters from <0> : return target creature card from your graveyard to the battlefield .


  1%|          | 1/100 [00:00<00:55,  1.77it/s]

[after preprocess]:activate only as a sorcery .
[after preprocess]:vigilance
[after preprocess]:<0> enters the battlefield with an <1> counter on it .
[after preprocess]:remove an <1> counter from <0> : it becomes an artifact creature until end of turn .
[after preprocess]:crew 3
[after preprocess]:tap any number of creatures you control with total power 3 or more : this vehicle becomes an artifact creature until end of turn .


  2%|▏         | 2/100 [00:01<00:51,  1.91it/s]

...

[after preprocess]:exile target non land permanent .
[after preprocess]:compleated
[after preprocess]:{u/p} can be paid with {u} or 2 life .
[after preprocess]:if life was paid , this planeswalker enters with two fewer loyalty counters .
[after preprocess]:+ 1 : until your next turn , up to one target creature gets - 3 / - 0 .
[after preprocess]:− 2 : target player mills three cards .
[after preprocess]:then if a graveyard has twenty or more cards in it , you draw three cards .
[after preprocess]:otherwise , you draw a card .
[after preprocess]:− x : target player mills three times x cards .


 46%|████▌     | 46/100 [00:18<00:26,  2.05it/s]

[after preprocess]:{1} , sacrifice <0> : draw a card .
[after preprocess]:{2} {g} , sacrifice <0> : target creature you control gets + 3 / + 3 and gains trample until end of turn .


 46%|████▌     | 46/100 [00:18<00:22,  2.45it/s]


KeyboardInterrupt: 